# 要約 
このJupyter Notebookは、「LMSYS - Chatbot Arena」コンペティションにおける不均衡データを扱うためのSMOTE（Synthetic Minority Over-sampling Technique）を適用し、機械学習モデルの訓練と推論プロセスを実施するものです。以下に、特に重要な側面を要約します。

### 問題の内容
- データセットには、モデルA、モデルB、引き分けの勝者を示すカラム（`winner_model_a`, `winner_model_b`, `winner_tie`）があり、これらのクラスには不均衡が存在します。この不均衡は、モデルの学習中にバイアスを引き起こし、特に少数派クラスに対する予測精度を低下させる恐れがあります。

### 解決手法
- SMOTE手法を利用して、トレーニングデータの不均衡を解消するため、合成データを生成し、全体のデータセットをリサンプリングします。これにより、各クラスに対するデータポイントの数を均等化します。

### ライブラリおよびツール
- **Pandas**: データの読み込み、結合、処理に使用。
- **NumPy**: 数値計算に使用。
- **imblearn**: SMOTEの実装を利用し、不均衡データの処理を行う。
- **Transformers**: `Gemma2ForSequenceClassification`モデルを使用して、応答を分類するためのモデル構築とトレーニングを行うために必要な機能を提供。
- **torch**: ディープラーニングモデルのトレーニングに使用。
- **datasets**: データセットの扱いや読み込みを容易にするライブラリ。
- **sklearn**: モデルの評価メトリック（log_loss, accuracy_score）を計算。

### プロセスの概要
1. トレーニングデータとテストデータを読み込む。
2. SMOTEを用いて不均衡なデータセットをリサンプリングする。
3. 読み込んだデータセットに対し、Gemmaモデルなどの事前学習済みのトランスフォーマモデルを設定し、トレーニング用のパラメータを構成する。
4. データをトークン化し、分類タスク用にモデルを訓練する。
5. テストデータに対して推論を行い、結果を整形して提出用CSVファイルとして保存する。

このノートブックは、機械学習モデルの不均衡データに対処するための重要な手法を示す良い例であり、最終的な提出ファイルでは、各モデルの勝者確率が求められています。

---


# 用語概説 
以下に、初心者がつまずきやすい専門用語の解説を示します。この解説は、実務経験のない方が特に馴染みのないかもしれない用語や、特定のノートブックに関連するドメイン知識に焦点を当てています。

1. **SMOTE (合成少数オーバーサンプリング技術)**:
   - 不均衡データに対処するために使われる手法で、少数派クラスのデータポイントを生成します。既存の少数派サンプル間の距離に基づいて合成サンプルを作成し、モデルがより多くの学習を行えるようにします。

2. **クラス不均衡**:
   - 機械学習において、あるクラスのデータポイントが他のクラスに比べて著しく少ない状態を指します。この状況は、モデルの性能に偏りをもたらし、少数派クラスの識別が難しくなることがあります。

3. **データコラトレーター (Data Collator)**:
   - モデルにデータを供給する際に使用されるもので、バッチ処理を行う際にデータを適切に整形する役割を果たします。異なる長さのシーケンスを適切にパディングしてから供給することが一般的です。

4. **LoRA (Low-Rank Adaptation)**:
   - 大規模なモデルのファインチューニングを効率的に行うための手法です。主にモデルの一部の層のみに変更を加えることで、全体のパラメータ数を大幅に減らしつつ性能を維持します。

5. **グラデーション蓄積 (Gradient Accumulation)**:
   - メモリ不足を回避するために、複数のバッチのグラデーションを蓄積してからモデルのパラメータを更新します。このプロセスにより、実際のバッチサイズをサポートして高性能を維持することができます。

6. **トークナイザー**:
   - テキストデータを数値（トークン）に変換するためのツールです。自然言語処理のモデルは、文字列を扱うことができないため、トークナイザーを用いて前処理を行い、数値的な形式に変換します。

7. **アテンションマスク (Attention Mask)**:
   - 注意機構を持つモデルで使用される、モデルが各トークンにどれだけ注意を払うかを示すマスクです。パディングされた部分を無視するために使用され、効率的な個別トークン処理を助けます。

8. **オプティマイザ (Optimizer)**:
   - 学習アルゴリズムにおいて、モデルのパラメータを更新して損失を最小化するための手法です。異なるオプティマイザにはそれぞれ異なる特徴があり、学習の収束速度や結果に影響を与えます。

9. **評価戦略 (Evaluation Strategy)**:
   - モデルの評価をどのように行うかを決める方針。例えば、エポックごとに評価を行う「epoch」戦略や、指定した間隔（保存ステップ）で評価を行う方法があります。

10. **トレーニング引数 (Training Arguments)**:
    - モデルの訓練時に使用される設定の集合で、エポック数、バッチサイズ、学習率など、訓練プロセスを調整するパラメータを含みます。

これらの用語は、初心者が機械学習や深層学習のプロセスを理解する上で重要です。特に、実務経験がない場合には、これらの概念が具体的にどのように関連するのかを把握することがキーとなります。

---


# トレインcsvの不均衡データのためのSMOTEを作成する



In [ ]:
%%time

import pandas as pd
import numpy as np

# トレインデータとテストデータ、サンプル提出ファイルを読み込む
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレインデータの読み込み
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')    # テストデータの読み込み
sample_submission = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出ファイルの読み込み

Kaggleの[このリンク](https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning?scriptVersionId=187770530)から手法をコピーしましたが、自分のバランスの取れたデータとgemmaモデルを使用して変更しました。



In [ ]:
%%time

# トレインデータの最初の51行を表示する
train.head(51)

In [ ]:
%%time

# テストデータの最初の2行を表示する
test.head(2)

In [ ]:
%%time

# データセットを結合する
combined_df = pd.concat([train, test], ignore_index=True)  # トレインデータとテストデータを結合

In [ ]:
%%time

from imblearn.over_sampling import SMOTE

# ターゲットカラムの欠損値を0で埋める（欠損値は勝利しなかったとみなす）
combined_df['winner_model_a'] = combined_df['winner_model_a'].fillna(0)
combined_df['winner_model_b'] = combined_df['winner_model_b'].fillna(0)
combined_df['winner_tie'] = combined_df['winner_tie'].fillna(0)

# SMOTEのためにターゲットカラムを一つのカラムに結合する
combined_df['winner'] = combined_df[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1)

# winnerカラムを数値にマッピングする
winner_mapping = {'winner_model_a': 0, 'winner_model_b': 1, 'winner_tie': 2}
combined_df['winner'] = combined_df['winner'].map(winner_mapping)

# 結合データセットにSMOTEを適用する
smote = SMOTE()
X = combined_df.drop(columns=['winner', 'winner_model_a', 'winner_model_b', 'winner_tie'])  # 特徴量を取り出す
y = combined_df['winner']  # ターゲットラベルを取り出す

# 簡単のために、テキストデータを単純な数値エンコーディングでエンコードする
X_encoded = X.apply(lambda col: col.astype('category').cat.codes if col.dtype == 'object' else col)

# SMOTEを適用する
X_resampled, y_resampled = smote.fit_resample(X_encoded, y)

# リサンプリングされたデータからデータフレームを作成する
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df['winner'] = y_resampled

# winnerカラムを元の形に戻す
inverse_winner_mapping = {v: k for k, v in winner_mapping.items()}
resampled_df['winner'] = resampled_df['winner'].map(inverse_winner_mapping)

# winnerカラムを元の3つのカラムに分割する
resampled_df['winner_model_a'] = (resampled_df['winner'] == 'winner_model_a').astype(int)
resampled_df['winner_model_b'] = (resampled_df['winner'] == 'winner_model_b').astype(int)
resampled_df['winner_tie'] = (resampled_df['winner'] == 'winner_tie').astype(int)

# 結合されたwinnerカラムを削除する
resampled_df = resampled_df.drop(columns=['winner'])

In [ ]:
%%time

# リサンプリングされたデータの最初の10行を表示する
resampled_df.head(10)

In [ ]:
%%time

# 欠損値を最頻値で埋める
combined_df['model_a'].fillna(combined_df['model_a'].mode()[0], inplace=True)
combined_df['model_b'].fillna(combined_df['model_b'].mode()[0], inplace=True)
combined_df['winner_model_a'].fillna(combined_df['winner_model_a'].mode()[0], inplace=True)
combined_df['winner_model_b'].fillna(combined_df['winner_model_b'].mode()[0], inplace=True)
combined_df['winner_tie'].fillna(combined_df['winner_tie'].mode()[0], inplace=True)

# 結合データフレーム内の欠損値をチェックする
missing_values = combined_df.isnull().sum()

# 欠損値を表示する
missing_values

In [ ]:
# 結合データフレームを新しいCSVファイルに保存する
combined_df.to_csv('combined_df.csv', index=False)

# なぜ不均衡（SMOTE）手法を作成したのか？

データセットはクラス不均衡に対処する必要があります。なぜなら、クラス不均衡なデータセットはモデル性能にバイアスをもたらす可能性があるからです。不均衡なデータセットで機械学習モデルをトレーニングすると、モデルは多数派クラスに偏りがちになり、少数派クラスへの性能が低下します。これは、モデルが新しい未学習データに対して一般化する能力に大きな影響を与える可能性があります。

このデータセットでは、勝者に関連するカラム（winner_model_a、winner_model_b、winner_tie）が不均衡です。以下は、問題の内訳です：

winner_model_a: このカラムは、モデルAが勝ったかどうかを示します。1のカウント（モデルAの勝利を示す）が他の勝者カラムと比較して極端に低いまたは高い場合、不均衡を生じます。

winner_model_b: このカラムは、モデルBが勝ったかどうかを示します。同様に、このカラムの1のカウントが他のカラムに対して不均衡である場合、問題を引き起こします。

winner_tie: このカラムは、結果が引き分けかどうかを示します。ここでの1の数が他の2つと比較して有意に少ない場合、不均衡が強調されます。

初期分析から、winner_tieカラムが他のカラムに比べて少ないインスタンスを持っていることが判明し、引き分けの発生頻度がモデルAまたはモデルBの勝利に対して不均衡であることを示しています。この不均衡は、モデルの学習プロセスを歪め、引き分けを予測する能力を低下させます。この不均衡をSMOTE（合成少数オーバーサンプリング技術）などの手法を使って解決することで、モデルがすべての可能な結果に対してよりバランスの取れた見方ができ、全体的なパフォーマンスが向上します。

# Gemmaモデルの使用



In [ ]:
%%time

# gemma-2はtransformers>=4.42.3から利用可能です
#!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [ ]:
%%time

import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

# 設定



In [ ]:
%%time

@dataclass
class Config:
    output_dir: str = "output"  # 出力ディレクトリ
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4ビット量子化されたgemma-2-9b-instructモデル
    max_length: int = 1024  # 最大シーケンス長
    n_splits: int = 5  # 分割数
    fold_idx: int = 0  # フォールドインデックス
    optim_type: str = "adamw_8bit"  # オプティマイザのタイプ
    per_device_train_batch_size: int = 2  # デバイスあたりのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8
    per_device_eval_batch_size: int = 8  # デバイスあたりの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # 合計42層中最初の16層にはアダプターを追加しない
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ数
    lora_r: int = 16  # LoRAのレイヤー数
    lora_alpha: float = lora_r * 2  # LoRAのアルファ値
    lora_dropout: float = 0.05  # LoRAのドロップアウト値
    lora_bias: str = "none"  # LoRAのバイアス設定
    
config = Config()  # Configクラスのインスタンスを生成

# トレーニング引数



In [ ]:
%%time

training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,  # 出力ディレクトリを上書きする
    report_to="none",  # ロギングの設定
    num_train_epochs=config.n_epochs,  # エポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスあたりのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # グラデーション蓄積ステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスあたりの評価バッチサイズ
    logging_steps=10,  # ロギングステップ
    eval_strategy="epoch",  # 評価戦略
    save_strategy="steps",  # 保存戦略
    save_steps=200,  # 保存ステップ
    optim=config.optim_type,  # オプティマイザのタイプ
    fp16=True,  # 16ビット浮動小数点を使用する
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ数
)

# LoRA設定



In [ ]:
%%time

lora_config = LoraConfig(
    r=config.lora_r,  # LoRAのレイヤー数
    lora_alpha=config.lora_alpha,  # LoRAのアルファ値
    # 自己注意層をターゲットにする
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],  # 変換する層の定義
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト値
    bias=config.lora_bias,  # LoRAのバイアス設定
    task_type=TaskType.SEQ_CLS,  # タスクタイプをシーケンス分類にする
)

# トークナイザーとモデルのインスタンス化



In [ ]:
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)  # 事前学習済みのトークナイザーを読み込む
tokenizer.add_eos_token = True  # テキストの末尾に<eos>トークンを追加
tokenizer.padding_side = "right"  # パディングを右側に設定

In [ ]:
%%time

model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,  # ラベル数を3に設定
    torch_dtype=torch.float16,  # 16ビット浮動小数点を使用
    device_map="auto",
)
model.config.use_cache = False  # キャッシュを使用しない設定
model = prepare_model_for_kbit_training(model)  # kビットトレーニング用のモデルを準備
model = get_peft_model(model, lora_config)  # LoRAモデルを取得
model  # モデルを表示

In [ ]:
%%time

model.print_trainable_parameters()  # 学習可能なパラメータを表示

# トレインファイルの読み込み



In [ ]:
%%time

ds = Dataset.from_csv("/kaggle/working/combined_df.csv")  # CSVファイルからデータセットを作成
ds = ds.select(torch.arange(100))  # デモ用に最初の100データを使用

In [ ]:
%%time

class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーを初期化
        self.max_length = max_length  # 最大長を設定
        
    def __call__(self, batch: dict) -> dict:
        # プロンプトを生成
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        # レスポンスAを生成
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        # レスポンスBを生成
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        # テキストを結合
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # トークン化
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels = []
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            # 勝者に応じたラベルを作成
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理して結合

In [ ]:
%%time

encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンス化
ds = ds.map(encode, batched=True)  # データセットにマッピングする

# メトリックloglossを計算する



In [ ]:
%%time

def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測結果を取得
    labels = eval_preds.label_ids  # ラベルを取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # ソフトマックスで確率に変換
    loss = log_loss(y_true=labels, y_pred=probs)  # loglossを計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度を計算
    return {"acc": acc, "log_loss": loss}  # 精度とloglossを辞書形式で返す

In [ ]:
%%time

# クロスバリデーションのためのインデックスセットを作成する
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

In [ ]:
%%time

# 訓練インデックスと評価インデックスを取得する
train_idx, eval_idx = folds[config.fold_idx]

# Trainerオブジェクトを設定する
trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),  # 訓練データセット
    eval_dataset=ds.select(eval_idx),    # 評価データセット
    compute_metrics=compute_metrics,      # メトリック計算関数
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # データコラトレーター
)
trainer.train()  # モデルの訓練を開始

# 推論



In [ ]:
%%time

# CUDAデバイスの数が2であることを確認する
assert torch.cuda.device_count() == 2

In [ ]:
%%time

@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'  # Gemmaモデルのディレクトリ
    lora_dir = '/kaggle/working/output/checkpoint-20'  # LoRAのチェックポイントディレクトリ
    max_length = 2048  # 最大シーケンス長
    batch_size = 4  # バッチサイズ
    device = torch.device("cuda")  # デバイスをCUDAに設定    
    tta = False  # テスト時のデータ拡張
    spread_max_length = False  # 各入力にmax_length//3を適用するか、結合された入力にmax_lengthを適用するか

cfg = Config()  # Configのインスタンスを作成

In [ ]:
%%time

# テストデータを読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

In [ ]:
%%time

def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

# テストデータのプロンプトとレスポンスを処理する
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))  # 最初の5行を表示する

In [ ]:
%%time

def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # プロンプトとレスポンスをトークン化する
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    if spread_max_length:
        # 最大長を各入力に適用する場合
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        # プロンプトとレスポンスを結合して最大長を適用する場合
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask  # トークン化されたIDとアテンションマスクを返す

In [ ]:
%%time

# Gemmaトークナイザーを読み込む
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True  # 終了トークンを追加
tokenizer.padding_side = "right"  # 右側パディングを設定

# テストデータのトークン化を実行
data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

# 拡張データを作成（response_aとresponse_bを入れ替える）
aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)

In [ ]:
%%time

# トークナイザーを使用して最初のデータをデコード
print(tokenizer.decode(data["input_ids"][0]))

In [ ]:
%%time

# 拡張データの最初のデータをデコード
print(tokenizer.decode(aug_data["input_ids"][0]))

In [ ]:
%%time

# GPU 0に基盤モデルを読み込む
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# GPU 1に基盤モデルを読み込む
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)

In [ ]:
%%time

from peft import PeftModel

# LoRAモデルを基盤モデルに追加
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)

In [ ]:
%%time

@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各勝者の確率を保持するリスト
    
    # データをバッチ処理で推論を行う
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))  # モデルで出力を取得
        proba = outputs.logits.softmax(-1).cpu()  # 確率に変換
        
        # 各勝者の確率をリストに追加
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win  # モデルAが勝つ確率
    df["winner_model_b"] = b_win  # モデルBが勝つ確率
    df["winner_tie"] = tie        # 引き分けの確率
    
    return df  # 結果のデータフレームを返す

In [ ]:
%%time

import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from transformers.data.data_collator import pad_without_fast_tokenizer_warning

st = time.time()

# 入力長でソートしてダイナミックパディングを最大限活用
data = data.sort_values("length", ascending=False)
# サブ1とサブ2のトークン数をほぼ同じにする
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

# スレッドプールを使って推論を並行処理する
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 確率を取得

print(f"elapsed time: {time.time() - st}")  # 経過時間を表示

In [ ]:
%%time

st = time.time()

if cfg.tta:  # テスト時のデータ拡張が有効な場合
    data = aug_data.sort_values("length", ascending=False)  # 入力長でソートして速度を向上
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)  # 結果を結合
    # TTAの順序を反転
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # 元の結果とTTAの結果を平均
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")  # 経過時間を表示

In [ ]:
%%time

# テストデータの順序に基づいて確率を整列させる
aligned_proba = proba[[result_df.index[result_df['id'] == id][0] for id in test['id']]]

# 予測を取得
test_pred_a = aligned_proba[:, 0]  # モデルAの予測
test_pred_b = aligned_proba[:, 1]  # モデルBの予測
test_pred_tie = aligned_proba[:, 2]  # 引き分けの予測

# 提出ファイルを準備する
submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': test_pred_a,  # モデルAの勝利確率
    'winner_model_b': test_pred_b,  # モデルBの勝利確率
    'winner_tie': test_pred_tie  # 引き分けの確率
})

# 提出ファイルを保存する
submission_path = '/kaggle/working/submission.csv'
submission.to_csv(submission_path, index=False)  # CSVファイルに保存

In [ ]:
submission.head()  # 提出ファイルの先頭を表示

ログロスが少ないほど、パフォーマンスの結果が良いことを示します。